# Pràctica 4 PLH - Rubén Álvarez Aragonés i Pol Pérez Prades

____

# Stuff

## Imports 

In [1]:
%pip install -r requirements.txt

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for fasttext
Failed to build fasttext
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
      c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\fasttext
      copying python\fasttext_module\fasttext\FastText.py -> build\lib.win-amd64-cpython-311\fasttext
      copying python\fasttext_module\fasttext\__init__.py -> build\lib.win-amd64-cpython-311\fasttext
      creating build\lib.win-amd64-cpython-311\fasttext\util
      copying python\fasttext_module\fasttext\util\util.py -> build\lib.win-amd6

In [2]:
import tensorflow as tf
import numpy as np
import nltk
import re
import spacy
from spacy.lang.ca.examples import sentences 
from gensim.models import word2vec
import torch
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')

spacy.cli.download("ca_core_news_sm")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Entrenament model Word2Vec

In [3]:
class Word2VecEmbedder:
    def __init__(self, corpus_path, corpus_size, load_model=False, model_path=None):
        if not load_model:
            self.corpus_path = corpus_path
            self.corpus_size = int(corpus_size * 2**30) if corpus_size else None  # Convert GB to bytes
            self.corpus = self.get_corpus(corpus_path)
            self.fit()
        else:
            try:
                self.load(model_path)
            except FileNotFoundError:
                print("Model not found. Please check the path.")
                return

    def get_corpus(self, corpus_path):
        with open(corpus_path, 'r', encoding='utf-8') as f:
            print("Reading corpus...")
            corpus = f.read(self.corpus_size) if self.corpus_size else f.read()
            print("Preprocessing corpus...")
            corpus = self.preprocess(corpus)  # Preprocess the corpus and tokenize it
        return corpus

    def fit(self, window_size=15, vector_size=300, min_count=10, workers=8, epochs=10):
        # Initialize the Word2Vec model with gensim
        print("Initializing Word2Vec model...")
        self.model = word2vec.Word2Vec(sentences=[self.corpus], vector_size=vector_size, window=window_size, min_count=min_count, workers=workers, epochs=epochs)
        print("Model training completed.")

    def save(self, model_path):
        # Save the model
        self.model.save(model_path)

    def load(self, model_path):
        # Load the model
        self.model = word2vec.Word2Vec.load(model_path)

    def preprocess(self, corpus):
        # Lowercase the corpus
        print("Lowercasing...")
        corpus = corpus.lower()
        
        # Remove special characters
        print("Removing special characters...")
        corpus = re.sub(r'[^a-záàéèíìóòúùñüç\s]', ' ', corpus)
        
        # Tokenize the corpus
        print("Tokenizing...")
        corpus = nltk.word_tokenize(corpus)
        
        # Eliminate last token (probably incomplete word)
        corpus = corpus[:-1]
        
        return corpus

    def get_embedding(self, word):
        # Get the embedding of a word
        try:
            return self.model.wv[word]
        except KeyError:
            print(f"Word '{word}' not in vocabulary.")
            return None

    def print_vocab(self):
        print("Vocabulary:", list(self.model.wv.index_to_key))
        
    

### 1. Model amb 100MB de dades

In [4]:
Word2Vec_model = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.1)

Reading corpus...
Preprocessing corpus...
Lowercasing...
Removing special characters...
Tokenizing...
Initializing Word2Vec model...
Model training completed.


In [5]:
Word2Vec_model.save('models/word2vec_model.bin')

In [6]:
Word2Vec_model.print_vocab()

Vocabulary: ['de', 'la', 'i', 'a', 'que', 'el', 'l', 'en', 'd', 'per', 'els', 'del', 'les', 'un', 'amb', 'una', 'va', 'ha', 'no', 'al', 'es', 'és', 'més', 'com', 'dels', 's', 'aquest', 'hi', 'si', 'o', 'se', 'tot', 'aquesta', 'han', 'fer', 'molt', 'però', 'ser', 'també', 'ja', 'li', 'seu', 'als', 'seva', 'n', 'cap', 'seus', 't', 'ens', 'pàgina', 'veure', 'sobre', 'catalunya', 'pel', 'després', 'estat', 'ho', 'perquè', 'varen', 'entre', 'on', 'fet', 'et', 'català', 'havia', 'lleida', 'tots', 'quan', 'qui', 'són', 'ara', 'm', 'van', 'des', 'fins', 'hem', 'això', 'era', 'partit', 'dia', 'fa', 'anys', 'dir', 'sempre', 'diputació', 'barcelona', 'què', 'part', 'altres', 'només', 'contra', 'durant', 'dos', 'sense', 'equip', 'any', 'voltants', 'interessa', 'recomanem', 'informació', 'primer', 'col', 'em', 'visitis', 'lloc', 'gran', 'hitler', 'tant', 'restaurant', 'cada', 'ni', 'president', 'encara', 'persones', 'e', 'té', 'així', 'estan', 'pot', 'està', 'llengua', 'mateix', 'vam', 'grup', 'aqu

In [7]:
Word2Vec_model.get_embedding("de")

array([-6.78541362e-01,  5.39869368e-01,  4.03966844e-01, -2.98002988e-01,
        3.77685241e-02, -6.49072886e-01,  1.13005388e+00,  2.64642787e+00,
        3.80026370e-01, -1.04509163e+00,  1.03648722e+00, -7.91770697e-01,
       -5.02870560e-01,  1.77563763e+00, -3.65493208e-01, -9.19002369e-02,
        8.21657002e-01,  6.21918738e-01, -2.37203926e-01, -1.98278868e+00,
        1.44154593e-01,  1.57724953e+00,  6.22737229e-01,  3.83101165e-01,
        4.17130500e-01,  7.41416931e-01,  6.66761875e-01,  4.43641275e-01,
       -2.34811790e-02, -1.18666637e+00,  4.22608882e-01, -3.85012984e-01,
        6.69768685e-03,  1.61847115e+00,  1.04314852e+00,  1.14948690e+00,
        1.10098040e+00, -3.70280504e-01,  1.27088404e+00,  1.04942369e+00,
       -2.46912226e-01, -3.05297613e-01,  7.34553635e-02, -1.76460469e+00,
        2.44311619e+00,  9.74084020e-01, -4.06426102e-01,  5.44636071e-01,
       -3.31721753e-01,  1.94776249e+00, -9.34404135e-01,  3.40923250e-01,
       -1.15695894e+00,  

In [8]:
Word2Vec_model.model.wv.most_similar("negre")

[('sabons', 0.22556228935718536),
 ('madero', 0.22364427149295807),
 ('defectius', 0.21694914996623993),
 ('desviar', 0.21588577330112457),
 ('creixi', 0.21561698615550995),
 ('abatuda', 0.20904815196990967),
 ('exempt', 0.20841456949710846),
 ('estalviar', 0.2059682458639145),
 ('esotèrics', 0.19971071183681488),
 ('israelianes', 0.19927068054676056)]

### 2. Model amb 500MB de dades

In [9]:
Word2Vec_model_500 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.5)

Reading corpus...
Preprocessing corpus...
Lowercasing...
Removing special characters...
Tokenizing...
Initializing Word2Vec model...
Model training completed.


In [10]:
Word2Vec_model.get_embedding("hola")

array([ 2.0775015e-03,  1.0567880e-03, -1.6255629e-03,  3.1482291e-03,
       -1.0828007e-03,  8.4958872e-04,  3.2786091e-03, -2.7655109e-03,
        3.2775463e-03,  6.6130323e-04,  1.2716241e-03,  1.1530495e-03,
       -4.9383880e-04,  1.9930000e-03, -1.8229814e-03,  4.5082212e-04,
        2.7142679e-03, -2.6555145e-03, -7.3076884e-04,  3.1427550e-03,
       -2.6453920e-03, -3.0769189e-03,  3.1929337e-03,  3.1773886e-03,
        2.0901405e-03, -3.0516200e-03,  2.4018362e-03,  1.8233713e-03,
       -2.6497131e-03, -2.0127888e-03, -8.6779514e-04,  2.7872846e-03,
        3.2888358e-03, -1.3961394e-04,  5.7863514e-04,  2.0927347e-03,
       -3.2404303e-03, -2.4493536e-04,  1.2174038e-03,  3.0606557e-03,
        1.9006491e-04,  2.1212928e-03, -8.0346031e-04, -1.3436433e-03,
        8.4563333e-04, -2.5548593e-03, -2.0761208e-03,  2.9200499e-03,
       -2.6264484e-03, -1.8010263e-03,  1.2320471e-03,  2.3446269e-03,
        1.1889736e-04,  2.8769842e-03, -2.2959744e-03,  2.7552882e-04,
      

### 3. Model amb 1GB de dades

In [ ]:
Word2Vec_model_1024 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 1)

In [ ]:
Word2Vec_model.get_embedding("hola")

### 4. Model amb totes les dades

In [ ]:
Word2Vec_model_full_data = Word2VecEmbedder('corpus\catalan_general_crawling.txt', None)

## Entrenament model de Similitud de Text Semàntic

### Imports

In [12]:
from importació_data import read_all_ts_data, reformat_data, create_corpus, preprocess, flattened_corpus_count, stopwords_cat
from importació_data import pair_list_to_x_y
from model_bàsic import build_and_compile_model_better
import tensorflow as tf
from model_bàsic import compute_pearson

Definim les stopwords del català i la funció del preprocessament del text, que les tindrà en compte a la hora de tokenitzar i natejar el text.

In [13]:
stpw_cat = stopwords_cat()
prepro = lambda x: preprocess(x, stpw_cat)

Per altra banda, llegim les dades i definim les variables més importants per a la creació dels models de similitud de text semàntic.
- Llegim totes les dades de text similarity dividint-les en train, test i val. 
- Reformatejem les dades per a que siguin l'estructura List[Tuple[str, str, float]]. 
- Definim el corpus i el diccionari amb totes les paraules.
- Creem un diccionari de python amb tots els indexs com a claus i amb la repetició de les paraules com a valor.

In [14]:
train, test, val = read_all_ts_data()
train, test, val = reformat_data(train, test, val)
corpus, dictionary = create_corpus(train, test, val, preprocess=prepro)
flat_corpus = flattened_corpus_count(corpus)

### Compartació amb diferents models de Word Embeddings

#### 1. One Hot

In [15]:
from onehot import map_one_hot

Un embedding OneHot té tamany igual a la llargada del diccionari. En la importació de dades ja hem eliminat les stopwords per reduïr la dimensió, però ara també eliminarem del embedding aquelles paraules que es repeteixen masses poques vegades o massa sovint. 

Per aconseguir això creem una llista que conté els indexs de les paraules que sí que utilitzarem i la passem com a argument a la funció *map_one_hot()*, per reduïr la dimensió del embedding.

In [16]:
# Eliminate from the dictionary the words that are repeated very few times or too many times
keys_preprocess = [index  for index in dictionary if flat_corpus[index] > 10 and flat_corpus[index] < 50]

Convertim les paraules a vectors OneHot amb la funció *map_one_hot()*. Aquesta funció crea un vector de zeros de la mida del diccionari i posa un 1 a la posició de la paraula en el diccionari.

In [17]:
mapped_one_hot_train = map_one_hot(train, dictionary, keys_preprocess)
mapped_one_hot_test = map_one_hot(test, dictionary, keys_preprocess)
mapped_one_hot_val = map_one_hot(val, dictionary, keys_preprocess)

Separem el X i Y

In [18]:
x_train_oh, y_train_oh = pair_list_to_x_y(mapped_one_hot_train)
x_val_oh, y_val_oh = pair_list_to_x_y(mapped_one_hot_val)
x_test_oh, y_test_oh = pair_list_to_x_y(mapped_one_hot_test)

Entrenem el model

In [19]:
batch_size_oh: int = 64
num_epochs_oh: int = 64

train_dataset_oh = tf.data.Dataset.from_tensor_slices((x_train_oh, y_train_oh))
train_dataset_oh = train_dataset_oh.shuffle(buffer_size=len(x_train_oh)).batch(batch_size_oh)

val_dataset_oh = tf.data.Dataset.from_tensor_slices((x_val_oh, y_val_oh))
val_dataset_oh = val_dataset_oh.batch(batch_size_oh)

In [20]:
embedding_size_oh = len(keys_preprocess)
model_oh = build_and_compile_model_better(embedding_size = embedding_size_oh)
tf.keras.utils.plot_model(model_oh, show_shapes=True, show_layer_activations=True, )
print(model_oh.summary())


You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 801)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 801)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 801)       │    642,402 │ input_layer[0][0… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 801)       │          0 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 801)       │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │     12,832 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         17 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 655,251 (2.50 MB)

 Trainable params: 655,251 (2.50 MB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
model_oh.fit(train_dataset_oh, epochs=num_epochs_oh, validation_data=val_dataset_oh)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.7322 - val_loss: 0.7500
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7212 - val_loss: 0.7449
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7142 - val_loss: 0.7424
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7054 - val_loss: 0.7387
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6815 - val_loss: 0.7280
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6250 - val_loss: 0.7228
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5158 - val_loss: 0.7340
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4111 - val_loss: 0.7561
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3244 - val_loss: 0.7734
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2701 - val_loss: 0.7871
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2326 - val_loss: 0.8030
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1949 - val_lo

Evaluem el model amb la partició de validació

In [22]:
print(f"Correlación de Pearson (train): {compute_pearson(model_oh, x_train_oh, y_train_oh)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_oh, x_val_oh, y_val_oh)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (train): 0.9560067996522469
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (validation): 0.17765272288170036


Provem el model amb la partició de test

In [23]:
print(f"Correlación de Pearson (test): {compute_pearson(model_oh, x_test_oh, y_test_oh)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (test): 0.21250305681247392


##### Conclusions de One-hot

Com podem observar a la hora de fer l'entrenament, el model fa molt *overfit*, així que no és capaç de generalitzar bé. Això és degut a que el model OneHot no té en compte la semàntica de les paraules, ja que cada paraula és tractada com a un element únic i no es té en compte el context en el que apareixen les paraules. Una altra raó per la que el model OneHot no és bó és perquè la mida dels vectors és molt gran, ja que la mida dels vectors és igual a la mida del diccionari, i això fa que el model sigui molt ineficient.

#### 2. Word2Vec preentrenats 

In [24]:
from word2vec_tf_idf import map_pairs_w2v

Llegim el model de Word2Vec preentrenat en català.

In [25]:
WORD_EMBEDDING_FILE = "C:/Users/ralva/Downloads/cc.ca.300.bin.gz"


USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.ca.300.bin'
    # wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')
else:
    from gensim.models import fasttext
    wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

##### 2.1 Mitjana

Convertim a vectors les frases amb la funció *map_w2v()*. Aquesta funció obté l'embedding de Word2Vec de cada paraula de la frase i en fa la mitjana.

In [26]:
mapped_w2v_mean_train = map_pairs_w2v(train, wv_model, dictionary=dictionary, preprocess=prepro)
mapped_w2v_mean_test = map_pairs_w2v(test, wv_model, dictionary=dictionary, preprocess=prepro)
mapped_w2v_mean_val = map_pairs_w2v(val, wv_model, dictionary=dictionary, preprocess=prepro)

Separem les dades en X i Y.

In [27]:
x_train_w2v_mean, y_train_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_train)
x_val_w2v_mean, y_val_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_val)
x_test_w2v_mean, y_test_w2v_mean = pair_list_to_x_y(mapped_w2v_mean_test)

Entrenem el model

In [28]:
batch_size_w2v_mean: int = 64
num_epochs_w2v_mean: int = 64

train_dataset_w2v_mean = tf.data.Dataset.from_tensor_slices((x_train_w2v_mean, y_train_w2v_mean))
train_dataset_w2v_mean = train_dataset_w2v_mean.shuffle(buffer_size=len(x_train_w2v_mean)).batch(batch_size_w2v_mean)

val_dataset_w2v_mean = tf.data.Dataset.from_tensor_slices((x_val_w2v_mean, y_val_w2v_mean))
val_dataset_w2v_mean = val_dataset_w2v_mean.batch(batch_size_w2v_mean)

In [29]:
embedding_size_w2v_mean = 300
model_w2v_mean = build_and_compile_model_better(embedding_size = embedding_size_w2v_mean)
tf.keras.utils.plot_model(model_w2v_mean, show_shapes=True, show_layer_activations=True)
print(model_w2v_mean.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 300)       │     90,300 │ input_layer_2[0]… │
│                     │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 300)       │          0 │ dense_3[0][0],    │
│                     │                   │            │ dense_3[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 300)       │          0 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │      4,816 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         17 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 1)         │          0 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,133 (371.61 KB)

 Trainable params: 95,133 (371.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
model_w2v_mean.fit(train_dataset_w2v_mean, epochs=num_epochs_w2v_mean, validation_data=val_dataset_w2v_mean)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.7254 - val_loss: 0.7327
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7092 - val_loss: 0.7098
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6831 - val_loss: 0.6914
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6477 - val_loss: 0.6713
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5944 - val_loss: 0.6529
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5315 - val_loss: 0.6417
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4575 - val_loss: 0.6291
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3769 - val_loss: 0.6161
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3151 - val_loss: 0.6273
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2688 - val_loss: 0.6225
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2249 - val_loss: 0.6285
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1831 - val_lo

Evaluem el model amb la partició de validació

In [31]:
print(f"Correlación de Pearson (train): {compute_pearson(model_w2v_mean, x_train_w2v_mean, y_train_w2v_mean)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_w2v_mean, x_val_w2v_mean, y_val_w2v_mean)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (train): 0.9828573064926769
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (validation): 0.38684400047991324


Provem el model amb la partició de test

In [32]:
print(f"Correlación de Pearson (test): {compute_pearson(model_w2v_mean, x_test_w2v_mean, y_test_w2v_mean)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (test): 0.3331129515366897


##### Conclusions de vectors preentrenats amb mitjana

Podem observar com amb el model Word2vec i la mitjana el model segueix fent sobreajust però funciona molt millor que el OneHot. Això és degut a que el model Word2Vec té en compte la semàntica de les paraules, ja que les paraules que tenen un significat similar tenen un embedding similar. Això fa que el model sigui capaç de generalitzar millor.

##### 2.2 Tf-idf

Definim el model de Tf-idf amb el corpus definit al inici.

In [33]:
from gensim.models import TfidfModel
modelo_tfidf = TfidfModel(corpus)

Fent servir la funció *map_w2v_tfidf()* s'obté l'embedding de Word2Vec de cada paraula de la frase i s'aplica Tf-idf per obtenir un vector de la frase.

In [34]:
mapped_w2v_tfidf_train = map_pairs_w2v(train, wv_model, dictionary=dictionary,tf_idf_model=modelo_tfidf, preprocess=prepro)
mapped_w2v_tfidf_test = map_pairs_w2v(test, wv_model, dictionary=dictionary,tf_idf_model=modelo_tfidf, preprocess=prepro)
mapped_w2v_tfidf_val = map_pairs_w2v(val, wv_model, dictionary=dictionary,tf_idf_model=modelo_tfidf, preprocess=prepro)

Separem les dades en X i Y.

In [35]:
x_train_w2v_tfidf, y_train_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_train)
x_val_w2v_tfidf, y_val_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_val)
x_test_w2v_tfidf, y_test_w2v_tfidf = pair_list_to_x_y(mapped_w2v_tfidf_test)

Entrenem el model

In [36]:
batch_size_w2v_tfidf: int = 64
num_epochs_w2v_tfidf: int = 64

train_dataset_w2v_tfidf = tf.data.Dataset.from_tensor_slices((x_train_w2v_tfidf, y_train_w2v_tfidf))
train_dataset_w2v_tfidf = train_dataset_w2v_tfidf.shuffle(buffer_size=len(x_train_w2v_tfidf)).batch(batch_size_w2v_tfidf)

val_dataset_w2v_tfidf = tf.data.Dataset.from_tensor_slices((x_val_w2v_tfidf, y_val_w2v_tfidf))
val_dataset_w2v_tfidf = val_dataset_w2v_tfidf.batch(batch_size_w2v_tfidf)

In [37]:
embedding_size_w2v_tfidf = 300
model_w2v_tfidf = build_and_compile_model_better(embedding_size = embedding_size_w2v_tfidf)
tf.keras.utils.plot_model(model_w2v_tfidf, show_shapes=True, show_layer_activations=True)
print(model_w2v_tfidf.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 300)       │     90,300 │ input_layer_4[0]… │
│                     │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 300)       │          0 │ dense_6[0][0],    │
│                     │                   │            │ dense_6[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 300)       │          0 │ lambda_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 16)        │      4,816 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 16)        │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1)         │         17 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 1)         │          0 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,133 (371.61 KB)

 Trainable params: 95,133 (371.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [38]:
model_w2v_tfidf.fit(train_dataset_w2v_tfidf, epochs=num_epochs_w2v_tfidf, validation_data=val_dataset_w2v_tfidf)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7237 - val_loss: 0.7331
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7044 - val_loss: 0.7138
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6845 - val_loss: 0.6952
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6526 - val_loss: 0.6786
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5968 - val_loss: 0.6575
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5353 - val_loss: 0.6407
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - loss: 0.4614 - val_loss: 0.6273
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3655 - val_loss: 0.6120
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3054 - val_loss: 0.6038
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2406 - val_loss: 0.6109
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2064 - val_loss: 0.6009
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1758 - val_lo

Evaluem el model amb la partició de validació

In [39]:
print(f"Correlación de Pearson (train): {compute_pearson(model_w2v_tfidf, x_train_w2v_tfidf, y_train_w2v_tfidf)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_w2v_tfidf, x_val_w2v_tfidf, y_val_w2v_tfidf)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Correlación de Pearson (train): 0.9820791386506129
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (validation): 0.4493208160812024


Provem el model amb la partició de test

In [40]:
print(f"Correlación de Pearson (test): {compute_pearson(model_w2v_tfidf, x_test_w2v_tfidf, y_test_w2v_tfidf)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (test): 0.3920015809377678


##### Conclusions vectors preentrenats amb Tf-idf

Podem observar com, fent us de Tf-idf, el model millora una mica més. Això és degut a que Tf-idf té en compte la freqüència de les paraules en el corpus, i això fa que les paraules que apareixen molt sovint tinguin un pes més baix. Això fa que el model sigui capaç de generalitzar millor. Tot i així el model segueix sobreajustant

#### 3. SpaCy

Importem tot allò necessari per a fer servir el model de SpaCy en català, inclòs el model preentrenat.

In [41]:
import spacy
from spacy_embed import map_spacy_embed

In [42]:
!python -m spacy download ca_core_news_md

     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.6/49.2 MB 13.1 MB/s eta 0:00:04
      --------------------------------------- 1.1/49.2 MB 12.0 MB/s eta 0:00:05
     - -------------------------------------- 1.8/49.2 MB 12.4 MB/s eta 0:00:04
     -- ------------------------------------- 2.5/49.2 MB 13.3 MB/s eta 0:00:04
     -- ------------------------------------- 3.4/49.2 MB 14.7 MB/s eta 0:00:04
     --- ------------------------------------ 4.5/49.2 MB 16.8 MB/s eta 0:00:03
     ---- ----------------------------------- 5.7/49.2 MB 18.4 MB/s eta 0:00:03
     ------ --------------------------------- 7.4/49.2 MB 19.7 MB/s eta 0:00:03
     ------- -------------------------------- 9.2/49.2 MB 22.7 MB/s eta 0:00:02
     --------- ----------------------------- 11.7/49.2 MB 28.5 MB/s eta 0:00:02
     ----------- --------------------------- 14.2/49.2 MB 38.6 MB/s eta 0:00:01
     -------------- ------------------------ 17.

Carreguem el model de SpaCy en català.

In [43]:
spacy_model = spacy.load("ca_core_news_md")

c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Passem les particions de train, test i val a vectors amb la funció *map_spacy()*. Aquesta funció utilitza el model de SpaCy per a convertir les frases a vectors.

In [44]:
mapped_spacy_train = map_spacy_embed(train, spacy_model)
mapped_spacy_test = map_spacy_embed(test, spacy_model)
mapped_spacy_val = map_spacy_embed(val, spacy_model)

Dividim les particions en X i Y.

In [45]:
x_train_sp, y_train_sp = pair_list_to_x_y(mapped_spacy_train)
x_val_sp, y_val_sp = pair_list_to_x_y(mapped_spacy_val)
x_test_sp, y_test_sp = pair_list_to_x_y(mapped_spacy_test)

Entrenem el model

In [46]:
batch_size_sp: int = 64
num_epochs_sp: int = 64

train_dataset_sp = tf.data.Dataset.from_tensor_slices((x_train_sp, y_train_sp))
train_dataset_sp = train_dataset_sp.shuffle(buffer_size=len(x_train_sp)).batch(batch_size_sp)

val_dataset_sp = tf.data.Dataset.from_tensor_slices((x_val_sp, y_val_sp))
val_dataset_sp = val_dataset_sp.batch(batch_size_sp)

In [47]:
embedding_size_sp = 300
model_sp = build_and_compile_model_better(embedding_size = embedding_size_sp)
tf.keras.utils.plot_model(model_sp, show_shapes=True, show_layer_activations=True, )
print(model_sp.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 300)       │     90,300 │ input_layer_6[0]… │
│                     │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 300)       │          0 │ dense_9[0][0],    │
│                     │                   │            │ dense_9[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 300)       │          0 │ lambda_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │      4,816 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 16)        │          0 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         17 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_7 (Lambda)   │ (None, 1)         │          0 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,133 (371.61 KB)

 Trainable params: 95,133 (371.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [48]:
model_sp.fit(train_dataset_sp, epochs=num_epochs_sp, validation_data=val_dataset_sp)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7266 - val_loss: 0.7404
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7163 - val_loss: 0.7326
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7043 - val_loss: 0.7222
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6890 - val_loss: 0.7142
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6734 - val_loss: 0.7051
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6516 - val_loss: 0.6999
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6224 - val_loss: 0.6939
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5958 - val_loss: 0.6824
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5618 - val_loss: 0.6788
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5336 - val_loss: 0.6656
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5041 - val_loss: 0.6523
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4738 - val_lo

Evaluem el model amb al partició de validation

In [49]:
print(f"Correlación de Pearson (train): {compute_pearson(model_sp, x_train_sp, y_train_sp)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_sp, x_val_sp, y_val_sp)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (train): 0.9441170210460957
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (validation): 0.3498764145329417


Provem el model amb la partició de test

In [50]:
print(f"Correlación de Pearson (train): {compute_pearson(model_sp, x_test_sp, y_test_sp)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (train): 0.26835966963377894


##### Conclusions de SpaCy

Utilitzant el model de SpaCy, el model segueix fent sobreajust, i funciona de manera molt similar al Word2Vec, però una mica pitjor. Això és degut a que el model de SpaCy també té en compte la semàntica de les paraules, ja que les paraules que tenen un significat similar tenen un embedding similar. Això fa que el model sigui capaç de generalitzar millor.

#### 4. RoBERTa 

In [51]:
from roberta_base import map_roberta_embed
import spacy

In [52]:
!python -m spacy download ca_core_news_trf

     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     ---------------------------------------- 0.1/457.1 MB 1.7 MB/s eta 0:04:37
     ---------------------------------------- 0.6/457.1 MB 5.1 MB/s eta 0:01:30
     ---------------------------------------- 1.1/457.1 MB 7.0 MB/s eta 0:01:06
     ---------------------------------------- 1.8/457.1 MB 8.6 MB/s eta 0:00:54
     --------------------------------------- 2.6/457.1 MB 10.4 MB/s eta 0:00:44
     --------------------------------------- 3.6/457.1 MB 12.1 MB/s eta 0:00:38
     --------------------------------------- 4.7/457.1 MB 13.8 MB/s eta 0:00:33
      -------------------------------------- 6.1/457.1 MB 15.7 MB/s eta 0:00:29
      -------------------------------------- 7.8/457.1 MB 17.8 MB/s eta 0:00:26
      -------------------------------------- 9.7/457.1 MB 20.1 MB/s eta 0:00:23
     - ------------------------------------ 12.1/457.1

Carreguem el model de RoBERTa preentrenat en català.

In [53]:
roberta_model = spacy.load("ca_core_news_trf")

Passem les particions de train, test i val a vectors amb la funció *map_roberta()*. Aquesta funció utilitza el model de RoBERTa per a convertir les frases a vectors.

In [54]:
mapped_roberta_train = map_roberta_embed(train, roberta_model)
mapped_roberta_test = map_roberta_embed(test, roberta_model)
mapped_roberta_val = map_roberta_embed(val, roberta_model)

Dividim les particions en X i Y.

In [55]:
x_train_roberta, y_train_roberta = pair_list_to_x_y(mapped_roberta_train)
x_val_roberta, y_val_roberta = pair_list_to_x_y(mapped_roberta_val)
x_test_roberta, y_test_roberta = pair_list_to_x_y(mapped_roberta_test)

Entrenem el model.

In [56]:
batch_size_roberta: int = 64
num_epochs_roberta: int = 64

train_dataset_roberta = tf.data.Dataset.from_tensor_slices((x_train_roberta, y_train_roberta))
train_dataset_roberta = train_dataset_roberta.shuffle(buffer_size=len(x_train_roberta)).batch(batch_size_roberta)

val_dataset_roberta = tf.data.Dataset.from_tensor_slices((x_val_roberta, y_val_roberta))
val_dataset_roberta = val_dataset_roberta.batch(batch_size_roberta)

In [57]:
embedding_size_roberta = 768
model_roberta = build_and_compile_model_better(embedding_size = embedding_size_roberta)
tf.keras.utils.plot_model(model_roberta, show_shapes=True, show_layer_activations=True)
print(model_roberta.summary())

You must install pydot (`pip install pydot`) for `plot_model` to work.


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 768)       │    590,592 │ input_layer_8[0]… │
│                     │                   │            │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_8 (Lambda)   │ (None, 768)       │          0 │ dense_12[0][0],   │
│                     │                   │            │ dense_12[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 768)       │          0 │ lambda_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 16)        │     12,304 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 16)        │          0 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1)         │         17 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_9 (Lambda)   │ (None, 1)         │          0 │ dense_14[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 602,913 (2.30 MB)

 Trainable params: 602,913 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

None


In [58]:
model_roberta.fit(train_dataset_roberta, epochs=num_epochs_roberta, validation_data=val_dataset_roberta)

Epoch 1/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.7277 - val_loss: 0.7421
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7180 - val_loss: 0.7324
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7062 - val_loss: 0.7160
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6830 - val_loss: 0.7027
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6553 - val_loss: 0.6951
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6191 - val_loss: 0.6858
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5954 - val_loss: 0.6782
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5688 - val_loss: 0.6639
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5409 - val_loss: 0.6500
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5164 - val_loss: 0.6398
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4744 - val_loss: 0.6205
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4561 - val_lo

Evaluar el model amb la partició de validació.

In [59]:
print(f"Correlación de Pearson (train): {compute_pearson(model_roberta, x_train_roberta, y_train_roberta)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_roberta, x_val_roberta, y_val_roberta)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (train): 0.9427521667979583
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (validation): 0.4714974301945399


Provar el model amb la partició de test.

In [60]:
print(f"Correlación de Pearson (test): {compute_pearson(model_roberta, x_test_roberta, y_test_roberta)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Correlación de Pearson (test): 0.2963311193928673


##### Conclusions Roberta

Veiem com el model de RoBERTa és un dels que millor funciona, ja que és el model més avançat i el que millor semàntica té. Això fa que el model sigui capaç de generalitzar millor.
Per altra banda, els seus vectors són més grans així que és més pesat d'executar que els altres models.

#### 5. RoBERTa fine-tuned

Instal·lem les llibreries necessàries

In [61]:
%pip install tf-keras
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Fem els imports de les funcions.

In [62]:
from roberta_fine_tuned import prepare_roberta_ft, compute_pearson_roberta_ft, x_y_split_roberta_ft
from transformers import pipeline, AutoTokenizer

Creem el model de RoBERTa fine-tuned.

In [63]:
model_roberta_ft = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer_roberta_ft = AutoTokenizer.from_pretrained(model_roberta_ft)
pipe_roberta_ft = pipeline('text-classification', model=model_roberta_ft, tokenizer=tokenizer_roberta_ft)

Separem en X i Y.

In [64]:
X_train_roberta_ft, y_train_roberta_ft = x_y_split_roberta_ft(train)
X_val_roberta_ft, y_val_roberta_ft = x_y_split_roberta_ft(val)
X_test_roberta_ft, y_test_roberta_ft = x_y_split_roberta_ft(test)

In [65]:
X = X_train_roberta_ft + X_val_roberta_ft + X_test_roberta_ft
y = y_train_roberta_ft + y_val_roberta_ft + y_test_roberta_ft

Obtenim els resultats del model

In [66]:
predictions = pipe_roberta_ft(prepare_roberta_ft(X, tokenizer_roberta_ft), add_special_tokens=False)

In [67]:
print(f"Correlación de Pearson (all data): {compute_pearson_roberta_ft(predictions, y)}")

Correlación de Pearson (all data): 0.890561552970006


In [68]:
print(f"Correlación de Pearson (test): {compute_pearson_roberta_ft(predictions[-len(y_test_roberta_ft):], y_test_roberta_ft)}")

Correlación de Pearson (test): 0.7527686784730463


##### Conclusions Roberta fine-tuned

Sens dubte el model de RoBERTa fine-tuned és el que millor funciona, ja que és el model més avançat i el que millor semàntica té. Això fa que el model sigui capaç de generalitzar millor.

## Models amb embeddings entrenables

In [69]:
REMAP_EMBEDDINGS: bool = True
USE_PRETRAINED: bool = True
MAX_LEN: int = 96

In [70]:
from trainable_embed_model import map_w2v_trainable, model_2, pair_list_to_x_y

Passem les particions de train, test i val a vectors amb la funció *map_w2v_trainable()*. Aquesta funció utilitza el model de Word2Vec per a convertir les frases a vectors.

In [71]:
mapped_train_trainable = map_w2v_trainable(wv_model, train, MAX_LEN, dictionary)
mapped_test_trainable = map_w2v_trainable(wv_model, test, MAX_LEN, fixed_dictionary=dictionary)
mapped_val_trainable = map_w2v_trainable(wv_model, val, MAX_LEN, fixed_dictionary=dictionary)

Partim en X i Y.

In [72]:
x_train_trainable, y_train_trainable = pair_list_to_x_y(mapped_train_trainable)
x_val_trainable, y_val_trainable = pair_list_to_x_y(mapped_val_trainable)
x_test_trainable, y_test_trainable = pair_list_to_x_y(mapped_test_trainable)

Reformatejem les dades per a que siguin l'estructura correcte.

In [73]:
batch_size_trainable: int = 64
num_epochs_trainable: int = 128

train_dataset_trainable = tf.data.Dataset.from_tensor_slices((x_train_trainable, y_train_trainable))
train_dataset_trainable = train_dataset_trainable.shuffle(buffer_size=len(x_train_trainable)).batch(batch_size_trainable)

val_dataset_trainable = tf.data.Dataset.from_tensor_slices((x_val_trainable, y_val_trainable))
val_dataset_trainable = val_dataset_trainable.batch(batch_size_trainable)

### Random embeddings

Creem el model amb embeddings aleatoris, sense cap tipus de preentrenament.

In [83]:
model_random = model_2(MAX_LEN, pretrained_weights=None, trainable=False, use_cosine=False)
model_random.summary()

c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'sequential_3' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_20      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 96, 16)    │     16,000 │ input_layer_19[0… │
│ (Embedding)         │                   │            │ input_layer_20[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_6         │ (None, 96)        │          0 │ input_layer_19[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_mask_6     │ (None, 96)        │          0 │ input_layer_19[0… │
│ (MyLayer_mask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_7         │ (None, 96)        │          0 │ input_layer_20[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_mask_7     │ (None, 96)        │          0 │ input_layer_20[0… │
│ (MyLayer_mask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 96, 1)     │        289 │ embedding_3[0][0… │
│ (Sequential)        │                   │            │ not_equal_6[0][0… │
│                     │                   │            │ embedding_3[1][0… │
│                     │                   │            │ not_equal_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 96, 1)     │          0 │ my_layer_mask_6[… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_7          │ (None, 96, 1)     │          0 │ my_layer_mask_7[… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_exp_6      │ (None, 96, 1)     │          0 │ sequential_3[0][… │
│ (MyLayer_exp)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_cast_6     │ (None, 96, 1)     │          0 │ get_item_6[0][0]  │
│ (MyLayer_cast)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_exp_7      │ (None, 96, 1)     │          0 │ sequential_3[1][… │
│ (MyLayer_exp)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_cast_7     │ (None, 96, 1)     │          0 │ get_item_7[0][0]  │
│ (MyLayer_cast)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_12         │ (None, 96, 1)     │          0 │ my_layer_exp_6[0… │
│ (Multiply)          │                   │            │ my_layer_cast_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_13         │ (None, 96, 1)     │          0 │ my_layer_exp_7[0… │
│ (Multiply)          │                   │            │ my_layer_cast_7[

 Total params: 16,578 (64.76 KB)

 Trainable params: 16,578 (64.76 KB)

 Non-trainable params: 0 (0.00 B)

Entrenem el model

In [ ]:
model_random.fit(train_dataset_trainable, epochs=num_epochs_trainable, validation_data=val_dataset_trainable)

Evaluem el model amb la partició de validació.

In [ ]:
print(f"Correlación de Pearson (train): {compute_pearson(model_random, x_train_trainable, y_train_trainable)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model_random, x_val_trainable, y_val_trainable)}")

Provem amb la partició de test.

In [ ]:
print(f"Correlación de Pearson (test): {compute_pearson(model_random, x_test_trainable, y_test_trainable)}")

### Word2Vec

Definim els pesos preentrenats. Cal destacar que el model de Word2Vec *wv_model* ja ha estat importat en la secció [2. Word2Vec preentrenats](#2-word2vec-preentrenats).

In [84]:
_pretrained_weights = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        _pretrained_weights = np.zeros(
            (len(dictionary.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in dictionary.token2id.items():
            if token in wv_model:
             _pretrained_weights[_id + 1] = wv_model[token]

            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        _pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        _pretrained_weights[1:, :] = wv_model.vectors

Creem el model amb embeddings de Word2Vec preentrenats.

In [85]:
model2 = model_2(MAX_LEN,pretrained_weights=_pretrained_weights, trainable=True, use_cosine=False)
model2.summary()

c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'sequential_4' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_24"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_23      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 96, 300)   │  3,878,100 │ input_layer_22[0… │
│ (Embedding)         │                   │            │ input_layer_23[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_8         │ (None, 96)        │          0 │ input_layer_22[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_mask_8     │ (None, 96)        │          0 │ input_layer_22[0… │
│ (MyLayer_mask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_9         │ (None, 96)        │          0 │ input_layer_23[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_mask_9     │ (None, 96)        │          0 │ input_layer_23[0… │
│ (MyLayer_mask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 96, 1)     │      4,833 │ embedding_4[0][0… │
│ (Sequential)        │                   │            │ not_equal_8[0][0… │
│                     │                   │            │ embedding_4[1][0… │
│                     │                   │            │ not_equal_9[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_8          │ (None, 96, 1)     │          0 │ my_layer_mask_8[… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_9          │ (None, 96, 1)     │          0 │ my_layer_mask_9[… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_exp_8      │ (None, 96, 1)     │          0 │ sequential_4[0][… │
│ (MyLayer_exp)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_cast_8     │ (None, 96, 1)     │          0 │ get_item_8[0][0]  │
│ (MyLayer_cast)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_exp_9      │ (None, 96, 1)     │          0 │ sequential_4[1][… │
│ (MyLayer_exp)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_cast_9     │ (None, 96, 1)     │          0 │ get_item_9[0][0]  │
│ (MyLayer_cast)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_16         │ (None, 96, 1)     │          0 │ my_layer_exp_8[0… │
│ (Multiply)          │                   │            │ my_layer_cast_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_17         │ (None, 96, 1)     │          0 │ my_layer_exp_9[0… │
│ (Multiply)          │                   │            │ my_layer_cast_9[

 Total params: 3,887,766 (14.83 MB)

 Trainable params: 3,887,766 (14.83 MB)

 Non-trainable params: 0 (0.00 B)

Entrenem el model.

In [86]:
model2.fit(train_dataset_trainable, epochs=num_epochs_trainable, validation_data=val_dataset_trainable)

Epoch 1/128


c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'my_layer_exp_9' (of type MyLayer_exp) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'my_layer_exp_8' (of type MyLayer_exp) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 1.7894 - val_loss: 0.8349
Epoch 2/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.8066 - val_loss: 0.7793
Epoch 3/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.7561 - val_loss: 0.7663
Epoch 4/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.7502 - val_loss: 0.7671
Epoch 5/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.7373 - val_loss: 0.7588
Epoch 6/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.7395 - val_loss: 0.7575
Epoch 7/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.7325 - val_loss: 0.7578
Epoch 8/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.7333 - val_loss: 0.7552
Epoch 9/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.7324 - val_loss: 0.7543
Epoch 10/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 0.7279 - val_loss: 0.7535
Epoch 11/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.7293 - val_loss: 0.7520
Epoch 12/128
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.7

Evaluem el model amb la partició de validació.

In [87]:
print(f"Correlación de Pearson (train): {compute_pearson(model2, x_train_trainable, y_train_trainable)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model2, x_val_trainable, y_val_trainable)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Correlación de Pearson (train): 0.04502294958392269
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Correlación de Pearson (validation): 0.059929202136506016


Provem amb la partició de test.

In [88]:
print(f"Correlación de Pearson (test): {compute_pearson(model2, x_test_trainable, y_test_trainable)}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Correlación de Pearson (test): 0.042564639119677804
